# ABM Data Prep
current version: September 19 2019

In [1]:
import pandas as pd
import os
import numpy as np

## Survey Data
Several lines of summary outputs are shown for reference.  In practice, these results were saved and combined with model data in excel to create the final datasets.  

In [2]:
surveydf = pd.read_csv('S:/AdminGroups/ResearchAnalysis/BER/ABM/FY18/_R_validation_notebook/data/survey/survey_trips_full.csv')

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (84,85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
purpose_dict = purpose_dict = {'discretionary':'Non-mandatory','eatingout':'Non-mandatory','escort':'Non-mandatory',
                'maintenance':'Non-mandatory','school':'Mandatory','shop':'Non-mandatory','university':'Mandatory',
                'visiting':'Non-mandatory','work':'Mandatory','work-based':'Non-mandatory'}

In [17]:
# Trip mode
surveydf.groupby('trip_mode').agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index()

,trip_mode,QUANTITY
0,Bike,2.748106e+05
1,Drive alone free,1.340106e+07
2,Drive alone pay,3.898055e+05
3,Drive to transit,1.774542e+05
4,School bus,7.478360e+05
5,Shared ride 2 free,6.531265e+06
6,Shared ride 2 pay,4.298708e+04
7,Shared ride 3+ free,5.499873e+06
8,Shared ride 3+ pay,2.087850e+04
9,Taxi,7.008415e+04


In [12]:
surveydf.groupby('trip_mode').agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).sum()

QUANTITY    3.172234e+07
dtype: float64

In [24]:
# mode x purpose
x = surveydf.groupby(['trip_mode','trip_purpose']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index()
x['Trips by purpose'] = x['trip_purpose'].map(purpose_dict)
x.head()
x.to_clipboard()

In [23]:
surveydf.groupby(['trip_mode','trip_purpose']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().sum()

trip_mode       BikeBikeBikeBikeBikeBikeBikeBikeBikeBikeDrive ...
trip_purpose    discretionaryeatingoutescortmaintenanceschools...
QUANTITY                                              3.14943e+07
dtype: object

In [31]:
# Trip Activities
surveydf.groupby('trip_purpose').agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index()

,trip_purpose,QUANTITY
0,discretionary,3.376778e+06
1,eatingout,2.535581e+06
2,escort,4.068983e+06
3,maintenance,4.689505e+06
4,school,2.490702e+06
5,shop,5.555766e+06
6,university,3.313851e+05
7,visiting,2.019572e+06
8,work,5.585900e+06
9,work-based,9.618548e+05


In [5]:
# trip purpose
q = x[['Trips by purpose','trip_purpose','QUANTITY']]
q.head()

,Trips by purpose,trip_purpose,QUANTITY
0,Non-mandatory,discretionary,56428.282232
1,Non-mandatory,eatingout,11712.567074
2,Non-mandatory,escort,5766.594686
3,Non-mandatory,maintenance,30578.692150
4,Mandatory,school,27745.147229


In [7]:
mode_dict = {'Bike':'Non-motorized','Drive alone free':'SOV',
                'Drive alone pay':'SOV','Drive to transit':'Transit access',
                'School bus':'HOV','Shared ride 2 free':'HOV','Shared ride 2 pay':'HOV',
                'Shared ride 3+ free':'HOV','Shared ride 3+ pay':'HOV',
                'Taxi':'HOV','Walk':'Non-motorized','Walk to transit':'Transit access'}

In [8]:
# trip mode 
t = surveydf.groupby('trip_mode').agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index()
t['Trips by mode'] = t['trip_mode'].map(mode_dict)
s = t[['Trips by mode','trip_mode','QUANTITY']]
s.head()

,Trips by mode,trip_mode,QUANTITY
0,Non-motorized,Bike,2.748106e+05
1,SOV,Drive alone free,1.340106e+07
2,SOV,Drive alone pay,3.898055e+05
3,Transit access,Drive to transit,1.774542e+05
4,HOV,School bus,7.478360e+05


In [33]:
# person type and mode - individual trips only
surveyone = surveydf[surveydf['JOINT_STATUS'] == 0]
surveyone.groupby(['trip_mode','type']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().head()

,trip_mode,type,QUANTITY
0,Bike,Child too young for school,4978.354652
1,Bike,Full-time worker,95009.707931
2,Bike,Non-worker,20288.397909
3,Bike,Part-time worker,26230.853185
4,Bike,Retired,9395.145395


In [15]:
surveyone.groupby(['trip_mode','type']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).sum()

QUANTITY    2.547227e+07
dtype: float64

In [34]:
# person type x purpose
surveyone.groupby(['trip_purpose','type']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().head()

,trip_purpose,type,QUANTITY
0,discretionary,Child too young for school,127848.677292
1,discretionary,Full-time worker,628028.797962
2,discretionary,Non-worker,341814.110224
3,discretionary,Part-time worker,169770.419759
4,discretionary,Retired,273192.208563


In [28]:
# household income
surveyhh = pd.read_csv('S:/AdminGroups/ResearchAnalysis/BER/ABM/FY18/_R_validation_notebook/data/survey/survey_hh.csv')
surveyhh = surveyhh[['SAMPN','HINCCAT2']]

surveyinc = surveydf.merge(surveyhh,on='SAMPN')

reclassdict = {1: 1, # less than 20k
               2: 1, # less than 35k
               3: 2, # less than 50k
               4: 3, # less than 60k
               5: 3, # less than 75k
               6: 4, # less than 100k
               7: 5} # more than 100k

surveyinc['HINCreclass'] = surveyinc.HINCCAT2.map(reclassdict)
surveyinc.groupby(['HINCreclass','trip_mode']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().head()

,HINCreclass,trip_mode,QUANTITY
0,1.0,Bike,4.670618e+04
1,1.0,Drive alone free,1.776067e+06
2,1.0,Drive alone pay,1.715668e+04
3,1.0,Drive to transit,1.526385e+04
4,1.0,School bus,1.275887e+05


In [37]:
def classdist(x):
    if x < 2.5:
        return 1
    elif x < 5:
        return 2
    elif x < 10:
        return 3
    elif x < 15:
        return 4
    elif x < 20:
        return 5
    elif x < 25:
        return 6
    elif x < 30:
        return 7
    elif x >= 30:
        return 8

In [40]:
# distance
distSkim = pd.read_csv(r'S:\AdminGroups\ResearchAnalysis\BER\ABM\FY18\validation\data\correspondance\distSkim.csv')
surveydf.loc[:,'ID'] = surveydf['ORIG_TAZ'].astype(str) + "-" + surveydf['DEST_TAZ'].astype(str)
distSkim.loc[:,'ID'] = distSkim['ORIG'].astype(str) + "-" + distSkim[' DEST'].astype(str)
#dist Skim
surveyDist = surveydf.merge(distSkim,how='left',left_on='ID',right_on="ID")
surveyDist['Dist (Miles)'] = surveyDist[' DIST'].map(classdist)
surveyDist.groupby(['Dist (Miles)','trip_purpose']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().head()

,Dist (Miles),trip_purpose,QUANTITY
0,1.0,discretionary,1.008462e+06
1,1.0,eatingout,7.433657e+05
2,1.0,escort,1.483105e+06
3,1.0,maintenance,1.248205e+06
4,1.0,school,1.125904e+06


In [122]:
# work trips by mode and distance
surveyDistwork = surveyDist[surveyDist['trip_purpose'] == 'work']

In [126]:
surveyDistwork.groupby(['Dist (Miles)','trip_mode']).agg({'EXPWGT':'sum'}).rename(columns={'EXPWGT':'QUANTITY'}).reset_index().head()

,Dist (Miles),trip_mode,QUANTITY
0,1.0,Bike,16336.509414
1,1.0,Drive alone free,342115.325300
2,1.0,Drive alone pay,3478.873516
3,1.0,Drive to transit,545.756139
4,1.0,School bus,169.028559


## Model Data

### combining individual and joint trips

In [41]:
output_dir = 'S:/AdminGroups/ResearchAnalysis/DataViz/abm_viz_report/modeldata_2010_calibration/raw/'
tap_dir = 'S:/AdminGroups/ResearchAnalysis/DataViz/abm_viz_report/modeldata_2010_calibration/raw/archive/'

In [42]:
expansion_size = 5

In [43]:
iTripscsv = pd.read_csv(os.path.join(output_dir,'indivTripData_1.csv')
                     ,usecols = [u'hh_id',u'person_id',u'tour_purpose'
                                 , u'trip_mode', u'tour_mode', u'orig_taz', u'dest_taz',u'orig_walk_segment'
                                 ,u'orig_purpose',u'dest_purpose',u'board_tap',u'orig_maz'
                                 , u'alight_tap'])

In [10]:
len(iTripscsv)

5198172

In [44]:
len(iTripscsv) * expansion_size

25990860

In [45]:
jTrips = pd.read_csv(os.path.join(output_dir,'jointTripData_1.csv')
                     ,usecols = [u'hh_id',u'tour_purpose',u'orig_maz',u'orig_walk_segment'
                                 , u'trip_mode', u'tour_mode', u'orig_taz', u'dest_taz'
                                 ,u'orig_purpose',u'dest_purpose',u'board_tap', u'alight_tap'
                                ,u'num_participants'])

In [46]:
len(jTrips)

1164928

In [47]:
len(jTrips) * expansion_size

5824640

In [48]:
tap = pd.read_csv(os.path.join(tap_dir,'tap_attributes.csv')
                 ,usecols=['tap_id','f_pnr_sp','if_pnr_sp','metra_rail','cta_rail','bus_exp','bus_local','taz09'])

In [49]:
tazgroups = pd.read_csv(r'S:\AdminGroups\ResearchAnalysis\BER\ABM\FY18\data\zoneGroups.txt')
distSkim = pd.read_csv(r'S:\AdminGroups\ResearchAnalysis\BER\ABM\FY18\validation\data\correspondance\distSkim.csv')

In [50]:
iTripsjTrips = iTripscsv.append(jTrips)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [51]:
iTrips_ = iTripsjTrips.loc[(iTripsjTrips['trip_mode'] > 0) & (iTripsjTrips['tour_mode'] > 0)]

In [52]:
len(iTrips_) * expansion_size

31812590

In [53]:
iTripso = iTrips_.merge(tap[['tap_id','metra_rail','cta_rail','bus_exp','bus_local','taz09']],
                        how='left',left_on='board_tap',right_on = 'tap_id')

In [54]:
iTripso.columns = iTrips_.columns.tolist() + ['tap_idb','metra_railb','cta_railb','bus_expb','bus_localb','taz09b']

In [55]:
iTrips = iTripso.merge(tap[['tap_id','metra_rail','cta_rail','bus_exp','bus_local','taz09']],
                       how='left',left_on='alight_tap',right_on = 'tap_id')

In [56]:
iTrips.columns = iTripso.columns.tolist() + ['tap_ida','metra_raila','cta_raila','bus_expa','bus_locala','taz09a']

In [57]:
iTrips.loc[:,'Model'] = expansion_size

iTrips.loc[:,'trip_purpose'] = 'na'
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['dest_purpose'] == 'Home',iTrips['orig_purpose'],iTrips['dest_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Discretionary','discretionary',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'EatingOut','eatingout',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Escort','escort',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Maintenance','maintenance',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Shop','shop',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'University','university',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Visiting','visiting',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'Work','work',iTrips['trip_purpose'])
iTrips.loc[:,'trip_purpose'] = np.where(iTrips['trip_purpose'] == 'work related','work-based',iTrips['trip_purpose'])

In [58]:
iTrips['trip_mode_code'] = iTrips['trip_mode']

In [59]:
iTrips[['trip_mode']] = iTrips[['trip_mode']].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14]
                     ,['Drive alone free','Drive alone pay'
                       ,'Shared ride 2 free','Shared ride 2 pay'
                       ,'Shared ride 3+ free','Shared ride 3+ pay'
                       ,'Walk','Bike','Walk to transit'
                       ,'Walk to transit','Drive to transit'
                       ,'Drive to transit','Taxi','School bus'])

In [60]:
# combines transit access modes - this is what we'll mostly use
iTrips[['trip_mode_transit']] = iTrips[['trip_mode_code']].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14]
                     ,['Drive alone free','Drive alone pay'
                       ,'Shared ride 2 free','Shared ride 2 pay'
                       ,'Shared ride 3+ free','Shared ride 3+ pay'
                       ,'Walk','Bike','Transit'
                       ,'Transit','Transit'
                       ,'Transit','Taxi','School bus'])

In [42]:
iTrips.to_csv("D:/dataviz/abm_model_data/itrips.csv")

### Trip Mode

### Data for csv1314_tripmode.csv

In [68]:
modeagg = iTrips.groupby('trip_mode_transit').agg({'hh_id':'count'}) * expansion_size

In [69]:
b = modeagg.transpose()

In [70]:
#b['sharedridepay'] = b['Shared ride 3+ pay'] + b['Shared ride 2 pay']

In [71]:
c = b.transpose().sort_values('hh_id',ascending=False).transpose()

In [72]:
#d = c.drop(['Shared ride 3+ pay', 'Shared ride 2 pay'],axis=1)

In [74]:
c

trip_mode_transit,Drive alone free,Shared ride 2 free,Shared ride 3+ free,Walk,Transit,School bus,Bike,Drive alone pay,Taxi,Shared ride 3+ pay,Shared ride 2 pay
hh_id,13271835,5680670,5146245,4420785,1626340,878250,329390,256355,99770,62510,40440


In [75]:
c.sum(axis=1)

hh_id    31812590
dtype: int64

### Data for csv9_mode_person.csv

In [76]:
# individual trips only

In [77]:
perm = pd.read_csv(output_dir + "personData_1.csv")

In [78]:
len(perm)

2048052

In [79]:
modwper = iTrips.merge(perm,on='person_id')

In [80]:
modwper.columns

Index(['alight_tap', 'board_tap', 'dest_purpose', 'dest_taz', 'hh_id_x',
       'num_participants', 'orig_maz', 'orig_purpose', 'orig_taz',
       'orig_walk_segment', 'person_id', 'tour_mode', 'tour_purpose',
       'trip_mode', 'tap_idb', 'metra_railb', 'cta_railb', 'bus_expb',
       'bus_localb', 'taz09b', 'tap_ida', 'metra_raila', 'cta_raila',
       'bus_expa', 'bus_locala', 'taz09a', 'Model', 'trip_purpose',
       'trip_mode_code', 'trip_mode_transit', 'hh_id_y', 'person_num', 'age',
       'gender', 'type', 'value_of_time', 'fp_choice', 'activity_pattern',
       'imf_choice', 'inmf_choice', 'walk_time_weight', 'walk_speed',
       'max_walk', 'user_class_work_walk', 'user_class_work_pnr',
       'user_class_work_knr', 'user_class_non_work_walk',
       'user_class_non_work_pnr', 'user_class_non_work_knr'],
      dtype='object')

In [82]:
len(modwper)

5198172

In [83]:
len(modwper) * expansion_size

25990860

In [84]:
a = (modwper.groupby(['trip_mode_transit','type']).agg({'trip_mode_transit':'count'}).rename(columns={'trip_mode_transit':'COUNT'}) * 5).reset_index()

In [85]:
b = a.groupby('trip_mode_transit').agg({'COUNT':'sum'}).reset_index()

In [86]:
b['type'] = 'All'

In [87]:
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [88]:
d = c.sort_values('COUNT',ascending=False).reset_index(drop=True)

In [92]:
d.head()

,COUNT,trip_mode_transit,type
0,13271835,Drive alone free,All
1,7586910,Drive alone free,Full-time worker
2,3880330,Walk,All
3,3074575,Shared ride 3+ free,All
4,2674540,Shared ride 2 free,All


### Data for csv10_mode_income

In [93]:
hh = pd.read_csv(output_dir + 'hhData_1.csv')
modwhh = iTrips.merge(hh,on='hh_id')

In [94]:
def classincome(x):
   if x < 35000:
       return 1
   elif x < 50000:
       return 2
   elif x < 75000:
       return 3
   elif x < 100000:
       return 4
   elif x >= 100000:
       return 5

In [95]:
modwhh['incomeclass'] = modwhh['income'].map(classincome)

In [96]:
a = (modwhh.groupby(['incomeclass','trip_mode_transit']).agg({'trip_mode_transit':'count'})*5).rename(columns={'trip_mode_transit':'COUNT'}).reset_index()

In [97]:
b = a.groupby('trip_mode_transit').agg({'COUNT':'sum'}).reset_index()
b['incomeclass'] = 'All'
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [98]:
c.sort_values('COUNT',ascending=False).head()

,COUNT,incomeclass,trip_mode_transit
56,13271835,All,Drive alone free
59,5680670,All,Shared ride 2 free
61,5146245,All,Shared ride 3+ free
65,4420785,All,Walk
1,3847285,1,Drive alone free


### Data for csv5_mode_purpose

In [99]:
a = (iTrips.groupby(['trip_mode_transit','trip_purpose']).agg({'trip_purpose':'count'}) * 5).rename(columns={'trip_purpose':'COUNT'}).reset_index()

In [100]:
b = a.groupby('trip_mode_transit').agg({'COUNT':'sum'}).reset_index()
b['trip_purpose'] = 'All'
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [101]:
c.sort_values('COUNT',ascending=False).head()

,COUNT,trip_mode_transit,trip_purpose
102,13271835,Drive alone free,All
105,5680670,Shared ride 2 free,All
107,5146245,Shared ride 3+ free,All
111,4420785,Walk,All
18,3929950,Drive alone free,work


### Trip Activities

### Data for csv15_16

In [102]:
a = (iTrips.groupby(['trip_purpose']).agg({'trip_purpose':'count'})*5).rename(columns={'trip_purpose':'COUNT'}).reset_index()

In [103]:
a.sort_values('COUNT',ascending=False).transpose().head()

,8,5,3,2,0,4,1,7,9,6
trip_purpose,work,shop,maintenance,escort,discretionary,school,eatingout,visiting,work-based,university
COUNT,5940075,5853935,4217855,3767495,3413480,2672310,2436550,1958215,1158190,394485


### Data for csv11

In [104]:
a = (modwper.groupby(['type','trip_purpose']).agg({'trip_purpose':'count'})*5).rename(columns={'trip_purpose':'COUNT'}).reset_index()

In [105]:
b = a.groupby('trip_purpose').agg({'COUNT':'sum'}).reset_index()
b['type'] = 'All'
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [106]:
c.head()

,COUNT,trip_purpose,type
0,136265,discretionary,Child too young for school
1,76010,eatingout,Child too young for school
2,290045,escort,Child too young for school
3,132505,maintenance,Child too young for school
4,317400,school,Child too young for school


### Data for csv12 and 12b

In [108]:
tazgroups = tazgroups.groupby('zone09').agg({'GROUP_':'first'})
tazgroups = tazgroups.reset_index()

In [109]:
iTripsTAZO = iTrips.merge(tazgroups,how='left',left_on='orig_taz',right_on="zone09")
iTripsTAZO.columns = iTrips.columns.tolist() + ['origzone09','origAreaGroup']
#iTripsTAZO_ = iTripsTAZO.drop_duplicates('index')

iTripsTAZOD = iTripsTAZO.merge(tazgroups,how='left',left_on='dest_taz',right_on="zone09")
iTripsTAZOD.columns = iTripsTAZO.columns.tolist() + ['destzone09','destAreaGroup']

iTripsTAZOD.loc[:,'ID'] = iTripsTAZOD['origzone09'].astype(int).astype(str) + "-" +\
                    iTripsTAZOD['destzone09'].astype(int).astype(str)

In [110]:
distSkim.loc[:,'ID'] = distSkim['ORIG'].astype(str) + "-" + distSkim[' DEST'].astype(str)

In [111]:
iTripsDist = iTripsTAZOD.merge(distSkim,how='left',left_on='ID',right_on="ID")

In [112]:
def classdist(x):
    if x < 2.5:
        return 1
    elif x < 5:
        return 2
    elif x < 10:
        return 3
    elif x < 15:
        return 4
    elif x < 20:
        return 5
    elif x < 25:
        return 6
    elif x < 30:
        return 7
    elif x >= 30:
        return 8

In [113]:
iTripsDist['distclass'] = iTripsDist[' DIST'].map(classdist)

In [114]:
a = (iTripsDist.groupby(['distclass','trip_purpose']).agg({'trip_purpose':'count'})*5).rename(columns={'trip_purpose':'count'}).reset_index()

In [115]:
b = a.groupby('trip_purpose').agg({'count':'sum'}).reset_index()
b['distclass'] = 'All'
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [116]:
c.head()

,count,distclass,trip_purpose
0,1071600,1,discretionary
1,687415,1,eatingout
2,1118715,1,escort
3,984370,1,maintenance
4,1294360,1,school


### data for csv7

In [117]:
wortrips = iTripsDist[iTripsDist['trip_purpose'] == 'work']

In [118]:
len(wortrips)

1188015

In [119]:
a = (wortrips.groupby(['distclass','trip_mode_transit']).agg({'trip_purpose':'count'})*5).rename(columns={'trip_purpose':'count'}).reset_index()

In [120]:
b = a.groupby('distclass').agg({'count':'sum'}).reset_index()
b['trip_mode_transit'] = 'All'
c = a.append(b, ignore_index=True)

C:\Users\sbuchhorn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [121]:
c.head()

,count,distclass,trip_mode_transit
0,31270,1,Bike
1,642555,1,Drive alone free
2,52295,1,Shared ride 2 free
3,9120,1,Shared ride 3+ free
4,980,1,Taxi
